In [133]:
import pandas as pd
import pickle

In [179]:
pickle_path = "../../sandbox/veronika_junkova/data_for_model.pck"

# Load the pickle file
with open(pickle_path, 'rb') as f:
    data = pickle.load(f)

print(data.columns)
data.drop("reviews_per_month", axis=1, inplace=True)
data.drop("min_rating", axis=1, inplace=True)
data.drop("bedrooms", axis=1, inplace=True)
data.drop("bedrooms", axis=1, inplace=True)


Index(['host_id', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'price', 'minimum_nights',
       'maximum_nights', 'has_availability', 'instant_bookable',
       'reviews_per_month', 'amenities_count', 'count_verifications',
       'seasonal_availability', 'season', 'min_rating', 'max_rating',
       'distance_from_city_centre'],
      dtype='object')


In [180]:
# categorical columns

#categorical_columns = data.select_dtypes(include=['object', 'category']).columns.astype(str)
categorical_columns = ['host_response_time', 'neighbourhood_cleansed', 'room_type', 'season']

print(categorical_columns)

['host_response_time', 'neighbourhood_cleansed', 'room_type', 'season']


In [181]:
# Linear Regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer

# Dummy encode categorical columns

dummy_df = pd.get_dummies(data, columns=categorical_columns)

# Split features (X) and target variable (y)
X = dummy_df.drop('price', axis=1)  # Features
y = dummy_df['price']  # Target variable

# Impute missing values in X
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

# Initialize and fit linear regression model
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred_linear = linear_regressor.predict(X_test)

# Calculate R-squared score
r2_linear = r2_score(y_test, y_pred_linear)
print("R2: {:.2f}".format(r2_linear))


R2: 0.02


In [182]:
# Catboost

from catboost import CatBoostRegressor

# Split features (X) and target variable (y)
X = data.drop('price', axis=1) 
y = data['price']

# Initialize and fit CatBoost regression model
catboost_regressor = CatBoostRegressor(iterations=1000,
                                       learning_rate=0.1,
                                       depth=7,
                                       loss_function='RMSE',
                                       verbose=0)
catboost_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred_catboost = catboost_regressor.predict(X_test)

# Calculate R-squared score
r2_catboost = r2_score(y_test, y_pred_catboost)
print("R2 (CatBoost): {:.2f}".format(r2_catboost))

R2 (CatBoost): 0.67


In [129]:
# Handle missing values in categorical columns

# Define the categories for the categorical columns
for col in categorical_columns:
    X[col] = X[col].astype('category').cat.add_categories('missing')


# Fill missing values in categorical features with a placeholder string
X[categorical_columns] = X[categorical_columns].fillna("missing")

print(df[])

In [130]:
# Catboost with Cross Validation

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# Define the number of folds for cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize CatBoostRegressor
catboost_regressor = CatBoostRegressor(iterations=1000, 
                                       learning_rate=0.1,
                                       depth=7,
                                       loss_function='RMSE',
                                       verbose=0,
                                       cat_features=['host_response_time', 'host_is_superhost', 'neighbourhood_cleansed',
                                       'room_type', 'has_availability', 'season'])

# Perform cross-validation
r2_scores = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Fit the model
    catboost_regressor.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = catboost_regressor.predict(X_test)
    
    # Calculate R-squared score
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Calculate the average R-squared score across all folds
avg_r2_score = sum(r2_scores) / len(r2_scores)
print("Average R2 score: {:.2f}".format(avg_r2_score))

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=4]=1.0 : cat_features must be integer or string, real number values and NaN values should be converted to string.